In [11]:
import pdb
import numpy as np
import pandas as pd
import os
import math
import matplotlib.pyplot as plt
import pylab as plt
from astropy.wcs import WCS
from astropy.io import fits
import readcol
from utils import shift_twod
from utils import dist_idl
from utils import loggen
from utils import gauss_kern
from lmfit import Parameters, minimize, fit_report
from simstack.simstack import stack_libraries_in_redshift_slices
from skymaps import Skymaps
from skymaps import Field_catalogs

%matplotlib inline  

In [12]:
popcolor=['blue','red','green','orange','black','grey','chocolate','darkviolet','pink',
          'magenta','dodgerblue','lavender','blue','red','green','orange','black','grey',
          'chocolate','darkviolet','pink','magenta','dodgerblue','lavender']

In [13]:
#Define Redshift Bins
z_lo = np.array([0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5])
z_hi = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0])
z_nodes = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0])
z_mid = (z_nodes[:-1] + z_nodes[1:])/2
nz = len(z_nodes) - 1

In [14]:
#Define Stellar Mass Bins
m_lo = np.array([8.5, 9.5,10.0,10.5,11.0])
m_hi = np.array([9.5,10.0,10.5,11.0,13.0])
m_nodes = np.array([8.5, 9.5,10.0,10.5,11.0,13.0])
m_mid = (m_nodes[:-1] + m_nodes[1:])/2
nm = len(m_nodes) - 1

In [15]:
#Decide which Maps to include
wv0 = np.array([1,1,1,1,1,1]) # This includes all maps
#wv0 = np.array([1,0,1,0,1,1]) # This one only includes the first, third, fifth, and sixth

## Map Directories and Names
## FWHM, Color Corrections also defined here if you use it  
# Need to Edit this for your own use 

In [16]:
indstack = np.where(wv0 == 1)
mapname=['_green'
         ,'_red'
         ,'_PSW'
         ,'_PMW'
         ,'_PLW'
         ,'_aztec'
        ]
library_keys =['pacs_green'
               ,'pacs_red'
               ,'spire_PSW'
               ,'spire_PMW'
               ,'spire_PLW'
               ,'aztec'
              ]
wavelength=[100,160,250,350,500,1100]
nwv = np.sum(wv0)
fwhm =[7.4, 11.3,18.1, 25.2, 36.6,18.]
efwhm=[6.7, 11.2,17.6, 23.9, 35.2,18.] # want to the measured effective FWHM later
color_correction=[23.58,23.82,1.018,0.9914,0.95615,1.0]

dir_spire_maps = '/data/maps_cats_models/maps/herschel/hermes/cosmos/'
dir_pacs_maps = '/data/maps_cats_models/maps/herschel/pacs/pep_cosmos_DR1/maps/'
dir_aztec_maps = '/data/maps_cats_models/maps/aztec/cosmos/forPeter/'
maps = [dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_green_flux.fits' 
        #dir_pacs_maps+'pep_COSMOS_green_Map.DR1.fits'
        ,dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_red_flux.fits' 
        #,dir_pacs_maps+'pep_COSMOS_red_Map.DR1.fits'
        ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PSW_flux.fits'
        ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PMW_flux.fits'
        ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PLW_flux.fits'
        #,dir_aztec_maps+'cosmos_jcmt_kscott20100925_map.fits'
        ,dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_aztec_flux.fits'
       ]
noises = [dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_green_noise.fits' 
          #dir_pacs_maps+'pep_COSMOS_green_Map.DR1.err.fits'
          ,dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_red_noise.fits' 
          #,dir_pacs_maps+'pep_COSMOS_red_Map.DR1.err.fits' 
          ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PSW_noise.fits'
          ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PMW_noise.fits'
          ,dir_spire_maps+'cosmos-cropped-uvista-hipe12_itermap_10_iterations_5.0_arcsec_pixels_PLW_noise.fits'
          #,dir_aztec_maps+'cosmos_jcmt_kscott20100925_weight.fits'
          ,dir_spire_maps+'cosmos-cropped-uvista_5.0_arcsec_pixels_aztec_noise.fits'
         ]#weight??
beams = ['/data/maps_cats_models/maps/herschel/pacs/pep_cosmos_DR1/PSF_observed_used/pep_COSMOS_green_Psf.DR1.fits'
         ,'/data/maps_cats_models/maps/herschel/pacs/pep_cosmos_DR1/PSF_observed_used/pep_COSMOS_red_Psf.DR1.fits'
         ,'/data/maps_cats_models/maps/beams/SPIRE/model/psw-model-beam_5_arcsec_pixels.fits'
         ,'/data/maps_cats_models/maps/beams/SPIRE/model/pmw-model-beam_5_arcsec_pixels.fits'
         ,'/data/maps_cats_models/maps/beams/SPIRE/model/plw-model-beam_5_arcsec_pixels.fits'
         ,'/data/maps_cats_models/maps/beams/aztec/aztec-beam_5_arcsec_pixels.fits'
        ]
sedstackbeams = [beams[0],beams[1],beams[2],beams[3],beams[4],beams[5]]

# Reading Maps into Objects, and Objects into Dictionaries

In [17]:
#Different libraries for Actual and Gaussian Approximated PSFs
sky_library_gaussians={}
sky_library_psfs={}

for t in indstack[0]:
    sky_library_gaussians[library_keys[t]] = Skymaps(maps[t],noises[t],efwhm[t],color_correction=color_correction[t])
    sky_library_psfs[library_keys[t]] = Skymaps(maps[t],noises[t],sedstackbeams[t],color_correction=color_correction[t])
    sky_library_gaussians[library_keys[t]].add_wavelength(wavelength[t])
    sky_library_gaussians[library_keys[t]].add_fwhm(efwhm[t])    
    sky_library_psfs[library_keys[t]].add_wavelength(wavelength[t])
    sky_library_psfs[library_keys[t]].add_fwhm(efwhm[t])

In [18]:
# Catalog Directories and Names

In [19]:
path_catalog = '/data/maps_cats_models/catalogs/UVISTA/'
file_catalog = 'UVISTA_final_v4.1_zCOSMOS_zEAZY_RFcolors_IDs_mass_AGN_SFRs_parentids.csv'

In [20]:
tbl = pd.read_table(path_catalog+file_catalog,sep=',')

In [21]:
uVista = Field_catalogs(tbl)

In [22]:
# This function creates a column 'sfg' based on its UVJ colors, where sf == 1 and qt == 0
uVista.separate_sf_qt()

In [ ]:
# This will seperate into sf/qt, redshift, and mass bins according to the z & m inputs
# Note, N z_nodes returns N-1 bins, etc.
uVista.get_sf_qt_mass_redshift_bins(z_nodes,m_nodes)

In [ ]:
#Stack in redshift bins, with Layers divided by Stellar Mass and Star-Forming/Quiescent Galaxies
pop = ['sf','qt']
npop=2
all_stacked_fluxes_psfs = np.zeros([nwv,nz,nm,npop])
all_luminosity_temp_psfs= np.zeros([nz,nm,npop,2])

for iz in range(nz):
    zn = z_nodes[iz:iz+2]
    uVista.get_sf_qt_mass_redshift_bins(zn,m_nodes)
    radec_m_z_p = uVista.subset_positions(uVista.id_z_ms)
    stacked_fluxes_psfs =  None
    n_sources_max = None
    
    stacked_fluxes_psfs = stack_libraries_in_redshift_slices(
        sky_library_gaussians,
        radec_m_z_p)
    
    #pdb.set_trace()
    args = radec_m_z_p.keys()
    for iwv in range(nwv):
        stacked_fluxes_wv = stacked_fluxes_psfs[str(wavelength[indstack[0][iwv]])]
        for j in range(nm):
            for k in range(npop):
                arg = 'z_'+str(zn[0])+'-'+str(zn[1])+'__m_'+str(m_nodes[j])+'-'+str(m_nodes[j+1])+'_'+pop[k]
                all_stacked_fluxes_psfs[iwv,iz,j,k] = stacked_fluxes_wv[arg.replace('.','p').replace('-','_')].value
    
    #pdb.set_trace()
    #PLOT
    plt.ylim([-1e-1,3e0])
    plt.ylim([1e-2,9e1])
    plt.xlim([90,510])
    plt.xlim([90,1150])
    plt.yscale('log')
    plt.ylabel('Flux Density [mJy]',fontsize=14)
    plt.xlabel('Wavelength [microns]',fontsize=14)
    for j in range(nm): 
        for k in range(npop):
            plt.plot(np.array(wavelength)[indstack],1e3*all_stacked_fluxes_psfs[:,iz,j,k],color=popcolor[npop*j+k],label=[pop[k],str(m_mid[j])])
            plt.legend()
        
    #pdb.set_trace()

In [ ]:
for iwv in range(len(wavelength)):
    plt.figure()
    for j in range(nm):
        
        #print 1e3*all_stacked_fluxes_psfs[iwv,:,j,0]
        plt.yscale('log')
        plt.title(str(wavelength[iwv]))
        plt.plot(z_mid,1e3*(all_stacked_fluxes_psfs[iwv,:,j,0]),'o')
        plt.plot(z_mid,1e3*(all_stacked_fluxes_psfs[iwv,:,j,0]))
    plt.show()